In [1]:
import csv
import math
from statistics import median, mode
from collections import Counter
from enum import Enum
from collections import Counter

In [2]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label


class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2


class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split#
    

In [15]:
node_type = NodeType.leaf

In [4]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes
        
class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition

class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

In [5]:
INFILE = "data/basketball.train.csv"
TEST_FILE = "data/basketball.test.csv"

OUTPUT_FILE = "data/basketball.result.csv"

In [16]:
ATTR = [Attribute("LOCATION", AttrType.cat),Attribute("W", AttrType.cat), Attribute("FINAL_MARGIN", AttrType.num),
        Attribute("SHOT_NUMBER", AttrType.num),Attribute("PERIOD", AttrType.cat),Attribute("GAME_CLOCK", AttrType.num),
        Attribute("SHOT_CLOCK", AttrType.num),Attribute("DRIBBLES", AttrType.num), Attribute("TOUCH_TIME", AttrType.num),
        Attribute("SHOT_DIST", AttrType.num),Attribute("PTS_TYPE", AttrType.cat),Attribute("CLOSE_DEF_DIST", AttrType.num),
        Attribute("Target", AttrType.target)]

In [7]:
IDX_TARGET = len(ATTR) - 1 

In [19]:
class DT(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.model = None  # decision tree model
        self.default_class = None  # default target class

    def load_data(self):
        with open(INFILE) as csvfile:
            self.data = []
            csvreader = csv.reader(csvfile, delimiter=',')
            for row in csvreader:
                rec = []
                for i in range(len(ATTR)):
                    val = row[i].strip()
                    # convert numerical attributes
                    if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
                        val = float(val)
                    rec.append(val)
                self.data.append(rec)
                # self.data.append([element.strip() for element in row])  # strip spaces
    
    def median(self,index):
        return median([self.data[idx][index] for idx in range(len(self.data))])
    
    def entropy(self, records):
        count = Counter([self.data[x][IDX_TARGET] for x in records]) # count the terget class in dictionary
        return sum([(-freq / len(records)) * math.log(freq / len(records), 2) for freq in count.values()])
    
    # given a set of records decides on wich attribute to split based on the biggest infogain value.
    def find_best_attr(self, attrs, records):

        entropy_p = self.entropy(records)  # parent's entropy
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split

            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            
            elif ATTR[a].type == AttrType.cat:  # categorical attribute
                #print("Categorical Attribute")
                split_mode = SplitType.multi
                split_cond = set([self.data[idx][a] for idx in range(len(self.data))])
                for condition in split_cond:
                    splits[condition] = [i for i in records if self.data[i][a] == condition ]

            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                #print("Numerical Attribute")
                split_mode = SplitType.bin
                split_cond = self.median(a)  # (i.e., if less or equal than this value)
                splits[str(split_cond)+"under"] = [i for i in records if self.data[i][a] <= split_cond ]
                splits[str(split_cond)+"over"] = [i for i in records if self.data[i][a] > split_cond ]
            
            #print("splits - >", splits)

            total = 0
            for split in splits :
                entropy_each = self.entropy(splits[split])
                total +=  entropy_each*(len(splits[split])/len(records))
                
            infogain = entropy_p - total

            splitting = Splitting(a, infogain, split_mode, split_cond, splits)
            splittings.append(splitting)

        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting
                
    # Adds a node to the decision tree.
    def add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,split_cond=None):
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # add it as a child of the parent node if there is such
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)
        return node_id
    
    # returns a decision tree.
    def id3(self, attrs, records, parent_id=None, value=None):
        
        # empty training set or empty set of attributes => create leaf node with default class
        if not records or not attrs:
            #print("empty training set")
            self.add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=self.default_class)
            return #returns nothing and stops function
        
        
        # if all records have the same target value => create leaf node with that target value
        same = all(self.data[idx][IDX_TARGET] == self.data[records[0]][IDX_TARGET] for idx in records)
        #print("same ->", same)
        if same:
            target = self.data[records[0]][IDX_TARGET]
            self.add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return
        
        
        # if amount of records is less than 10000
        if len(records) < 20000 : 
            default_class = Counter([self.data[x][IDX_TARGET] for x in records]).most_common(1)[0][0]
            self.add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=default_class)
        
            
        # find the attribute with the largest gain
        splitting = self.find_best_attr(attrs, records)
        
        # add node
        node_id = self.add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
 
        # TODO call tree construction recursively for each split
        attrs.remove(splitting.attr)        
        for branch in splitting.splits:
            self.id3(attrs, splitting.splits[branch], node_id, ATTR[splitting.attr].label)
        return
        
    
    def build_model(self):
        self.load_data()
        self.model = []  # holds the decision tree model, represented as a list of nodes
        self.default_class = Counter([x[IDX_TARGET] for x in self.data]).most_common(1)[0][0]
        amount_of_records = len(self.data)
        self.id3(set(range(len(ATTR) - 1)), list(range(amount_of_records)))
        
    def apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
            if ATTR[node.val].type == AttrType.cat : 
                node_index = node.children[list(node.split_cond).index(record[node.val])]
            else :
                
                if float(record[node.val]) <= node.split_cond:
                    node_index = node.children[0]
                else :
                    node_index = node.children[1]
            node = self.model[node_index]
        return node.val
    
    def create_output(self):
        temp_list = []
        with open(TEST_FILE) as f :
            csvreader = csv.reader(f,delimiter=',')
            cnt = 0
            for row in csvreader :
                cnt +=1
                temp_list.append([cnt,self.apply_model(row)])
        file = open(OUTPUT_FILE,"w")
        file.write("Id,Target\n") 
        for row in temp_list:
            file.write("%s,%s\n" % (row[0], row[1]))
        file.close() 



In [20]:
def main():
    dt = DT()
    dt.build_model()
    dt.create_output()

if __name__ == "__main__":
    main()

### Code Practice 

In [ ]:
# checking if list is empty
rec = [1,2,3,4]
if not rec:
    print("hala")

In [ ]:
# Default argument principle
def macs(sulo, ahuto = 5):
    return sulo + ahuto

print(macs(4,6))

In [ ]:
#empty return statement
def testReturn(a,b):
    if a>b:
        return
    print(a+b)

print(testReturn(5,4))

In [ ]:
#Enum class
class Days(Enum):
    Sun = 1
    Mon = 2
    Tue = 3
    
print (Days.Mon)
print (repr(Days.Sun))
print (type(Days.Mon))
print (Days.Tue.name)

In [ ]:
#List comprehensions
data = [[1,2,3,"miss"],[4,5,6,"score"],[7,8,9,"score"],[10,11,12,"score"]]
records = [0,1,2,3]

IDX_TARGT = 3
count = Counter([data[x][IDX_TARGT] for x in records])
print(count)